In [ ]:
#Get JOURNAL NAME FROM PMID

import requests
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import time
import os
import lxml


######################CHANGE DIRECTORY BASED ON WHERE UR FILES ARE.########################
location = os.listdir(r'Directory of files you want to read')
######################CHANGE DIRECTORY BASED ON WHERE UR FILES ARE.########################






length = len(location)
pmids = np.array([])


#Looks up PMID at Pubmed at finds associated Journal Name
def journals(x):
    journal_name = np.array([])
    pmids = np.array([])
    url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id='
    api_key = '&examplePubmedApiKey'
    for i in range(x, length):
######################################################CHANGE 'FILES' BASED ON WHERE Your FILES ARE LOCATED.#################
        files = os.listdir(r'Directory of files you want to read\\')[x]
######################################################CHANGE 'FILES' BASED ON WHERE Your FILES ARE LOCATED.#################
        
        print('The file I am reading is: ' + files)
    
###############################CHANGE TEXt TO THE LOCATION THAT You WANT TO READ FILES FROM######################    
        csv = pd.read_excel(io=r'Directory of files you want to read\\' + files, sheet_name='Sheet1')
###############################CHANGE TExt TO THE LOCATION THAT You WANT TO READ FILES FROM######################  
        
        csv = csv.dropna(subset=['pmid'])
    
#i = author name
#x = pmid    
        for journal,pmid in csv.get(['journal', 'pmid']).itertuples(index=False):
            name_to_string = str(journal)
            string_to_int = int(pmid)
            if name_to_string == 'nan':
            #search for author name using pmid
                try:
                    rand_num = np.random.uniform(0.2, .4)
                    time.sleep(rand_num) #random delay between 0.2 - 0.4 seconds so we dont hit api call limit
                    page = requests.get(url + str(pmid) + api_key)
                    print(page.status_code)
                    print('The file I am reading is: ' + files)
                    soup = bs(page.text, "lxml-xml")
                    getting_closer = soup.find("Item", {"Name":"FullJournalName"}).get_text(strip=True)
                    journal_name = np.append(journal_name, getting_closer)
                    if len(journal_name) >= len(csv.get('pmid')): #if length of journal array is equal or larger than the number of PMIDs in starting csv it will create export sheet
                        author_df = pd.DataFrame().assign(Journal= journal_name)
######################################EXPORT LOCATION OF FINISHED SHEETS in CSV FORM#####################################
                        author_df.to_csv(r'Directory you want to save finished files on\\' + files + 'JOURNALS.csv')
                        csv['Journals'] = journal_name
################################################EXPORT LOCATION OF FINISHED SHEETS IN XLSX FORMAT#########################
                        csv.to_excel(r'Directory you want to save finished files on\\' + files, index=False)
                        x = x + 1
                        #journals(x) uncomment here to iterate through the folder with multiple files
                
                except: #if it cant find a journal name it will append NA to the array
                    journal_name = np.append(journal_name, "NA")
                    
            else: #if csv does not have empty journal column it will append data from journal column saving api calls
            #add original author name
                journal_name = np.append(journal_name, journal)
                #print(len(authors))
                if len(journal_name) >= len(csv.get('pmid')):  #if length of journal array is equal or larger than the number of PMIDs in starting csv it will create export sheet
                    author_df = pd.DataFrame().assign(Journal= journal_name)
######################################EXPORT LOCATION OF FINISHED SHEETS in CSV FORM#####################################
                    author_df.to_csv(r'Directory you want to save finished files on\\' + files + 'JOURNALS.csv')
                    csv['Journals'] = journal_name
                    csv.to_excel(r'Directory you want to save finished files on\\' + files, index=False)
                    x = x + 1
                    #journals(x) uncomment here to iterate through the folder with multiple files

In [ ]:
journals(0) #change the number based on which file you want to start with. 0 starts with first file in the directory